In [60]:
from toqito.channels import partial_trace
from qutip import *
from PIL import Image
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import linalg
import math
import matplotlib.pyplot as plt
from scipy import optimize
import random
from math import *
import pandas as pd
import random
from mpl_toolkits.mplot3d import Axes3D
from sys import stdout
from tqdm import tqdm
from tqdm import trange
from scipy.linalg import fractional_matrix_power

In [61]:
import time
from datetime import datetime as dt 
date = dt.now()
printdate = date.strftime('%Y%m%d_%H%M%S')
print(date)

2023-06-21 22:13:14.243209


In [62]:
def state_fidelity(rho_1, rho_2): #fidelity
        if np.shape(rho_1) != np.shape(rho_2):
            print("Dimensions of two states do not match.")
            return 0
        else:
            sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
            fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
            return np.real(fidelity)

In [63]:
def UO(B1,B2,a,D1,D2):
    i   = 1j
    gamma = 2*pi*2.8
    D     = 2870
    UA = [[(B2**2+B1**2*cos(a))/(B1**2+B2**2), -i*B1*(e**(-i*D1))*sin(a)/sqrt(B1**2+B2**2), ((-1+cos(a))*B1*B2*(e**(-i*(D1-D2))))/(B1**2+B2**2)],
            [-i*B1*(e**(i*D1))*sin(a)/sqrt(B1**2+B2**2), cos(a), -i*B2*(e**(i*D2))*sin(a)/sqrt(B1**2+B2**2)],
            [((-1+cos(a))*B1*B2*e**(i*(D1-D2)))/(B1**2+B2**2), -i*B2*(e**(-i*D2))*sin(a)/sqrt(B1**2+B2**2), (B1**2+B2**2*cos(a))/(B1**2+B2**2)]]
    return UA

## Define dimension, pauli matrices
i   = 1j #1j
sx  = 1/sqrt(2)*np.array([[0, 1, 0],[1, 0, 1], [0, 1, 0]])
sy  = 1/sqrt(2)/i*np.array([[0, 1, 0], [-1, 0, 1],[0, -1, 0]])
sz  = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
#sz  = [1, 0, 0; 0, -1, 0; 0, 0, 0]
I   = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Rotation matrix projected into 2 level system
Sxp  = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])
Sxm  = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])
Syp  = 1/i*np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 0]])
Sym  = 1/i*np.array([[0, 0, 0], [0, 0, 1], [0, -1, 0]])
Szp  = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]])

#Gellman matrix
Sx  = np.array([[0, 0, 1],[0, 0, 0], [1, 0, 0]])
Sy  = np.array([[0, 0, -i],[0, 0, 0], [i, 0, 0]])
Sz  = np.array([[1, 0, 0],[0, 0, 0], [0, 0, -1]])

# Pauli basis for 13C nuclear spin
Ix  = 1/2*np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]])   
Iy  = 1/2/i*np.array([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
Iz  = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])


## Define sweep parameters
Sweep = 1001
N = Sweep
B = 403 #[G] magnetic field

T = 5; # sweep tau [us]
t = np.linspace(0,T,N)
n = 32; # number of pi pulses

## Define gate operations
# Single Q ms=+1
U090xp = UO(1,0,pi/4,0,0)
U090xmp = UO(1,0,-pi/4,0,0)
U090yp = UO(1,0,pi/4,pi/2,0)
U090ymp = UO(1,0,-pi/4,pi/2,0)
U180xp = UO(1,0,pi/2,0,0)
U180xmp = UO(1,0,-pi/2,0,0)

#Single Q ms=-1
U090xm = UO(0,1,pi/4,0,0)
U090xmm = UO(0,1,-pi/4,0,0)
U180xm = UO(0,1,pi/2,0,0)
U180xmm = UO(0,1,pi/2,0,0)

# Define initial state of the system

irho_p = np.array([[1,0,0],[0,0,0],[0,0,0]]) #[0,0,0;0,0,0]

irho_m = np.array([[0,0,0],[0,0,0],[0,0,1]]) #[0,0,0;0,0,1]

irho_z = np.array([[0,0,0],[0,1,0],[0,0,0]]) #[0,1,0;0,0,0]

irho_mix = np.array([[1/2,0,0],[0,1/2,0],[0,0,0]]) #[1/2,0,0;0,1/2,0;0,0,0]

irho_Z = np.array([[0,0,0],[0,0,0],[0,0,1]]) #target state

irho_MIX = np.array([[1/2,0,0],[0,0,0],[0,0,1/2]])

In [64]:
arr1 = []

In [65]:
irho = np.kron(irho_z,irho_MIX) #initial state
print(irho)

[[0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]]


In [66]:
trace = [1, 1, 0, 100, 100, 1000]
vvv = [0, 0, 0, 0]
normalxyz = [0, 0, 0]

#for making 13C nuclear random dataset
gammaN = 2*pi*1.071e-3 #[MHz/G]
# Al    = 2*pi * random.uniform(0.05, 0.8) #[MHz] # A_|| hyperfine term
# Ap = 2*pi* random.uniform(0.05, 0.3) #[MHz] # A_per hyperfine term

Al = 1.1876071666049706
Ap = 1.3050296350856783

#Initialization
rho_0 = (np.kron(U090xp,I))@irho@((np.kron(U090xp,I)).conj().T) # superposition state on NV

Sa= []

ham = Al*np.kron(sz,Iz) + Ap*np.kron(sz,Ix) + B*gammaN*np.kron(I,Iz) # Hamiltonian
eigvals = np.linalg.eigh(ham)[0]            # diagonalizing the Hamiltonian 여기서부터 문제 
eigvecs = -1*np.linalg.eigh(ham)[1]         # eigenvectors
E = np.diag(eigvals)                        # exponent of eigenvalues
U_H= eigvecs.conj().T                       # unitary matrix formed by eigenvectors

In [67]:
# udf = pd.DataFrame(U_H)
# udf

partial_trace(U_H, 1)

array([[ 1.06242308, -1.        , -1.19742667],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [68]:
rho1 = np.kron(U090yp,I)@irho@(np.kron(U090yp,I).conj().T)                              # Ry 90도
df1 = pd.DataFrame(rho1)
print(partial_trace(rho1, 1))

[[0.5-1.00477575e-34j 0. +0.00000000e+00j 0. +0.00000000e+00j]
 [0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j]
 [0. +0.00000000e+00j 0. +0.00000000e+00j 0.5-1.00477575e-34j]]


In [69]:
vari=[1.4120767867009363,22.0,0.2633900964243493,22.0]  #초기값

In [70]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E* vari[0]/2)@U_H)                                  # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E* vari[0])@U_H)                                     # for tau
rho2=U_e2@rho1@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[1])):                                                # N과 tau를 N개 생성
    rho2 = U_e@np.kron(U180xp,I) @ rho2 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho3 = U_e2 @ np.kron(U180xp,I) @ rho2 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

In [71]:
# df2 = pd.DataFrame(rho2)
# df2 = pd.DataFrame(rho3)
# df2
print(partial_trace(rho3, 1))

[[ 5.00000000e-01-2.02866410e-16j  0.00000000e+00+0.00000000e+00j
   4.84429831e-17+7.70233606e-17j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j]
 [-2.47416534e-17-1.07368023e-16j  0.00000000e+00+0.00000000e+00j
   5.00000000e-01-9.44539817e-17j]]


In [72]:
#for e Rx(pi/2)
rho4 = np.kron(U090xp,I)@rho3@(np.kron(U090xp,I).conj().T)                              # Rx 90도
df4 = pd.DataFrame(rho4)
print(partial_trace(rho4, 1))

[[ 5.00000000e-01-2.02817836e-16j  0.00000000e+00+0.00000000e+00j
   2.77555756e-17+7.80625564e-17j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j]
 [-2.77555756e-17-1.07552856e-16j  0.00000000e+00+0.00000000e+00j
   5.00000000e-01-9.46401797e-17j]]


In [80]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E*vari[2]/2)@U_H)                                   # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E*vari[2])@U_H)                                      # for tau/2
rho5=U_e2@rho4@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[3])):                                                # N과 tau를 N개 생성
    rho5 = U_e@np.kron(U180xp,I) @ rho5 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho6 = U_e2 @ np.kron(U180xp,I) @ rho5 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

# df6 = pd.DataFrame(rho6)
# print(df6)
print(partial_trace(rho6, 1))

[[0.52123879-2.10902395e-16j 0.        +0.00000000e+00j
  0.00384311+1.81195639e-03j]
 [0.        +0.00000000e+00j 0.        +0.00000000e+00j
  0.        +0.00000000e+00j]
 [0.00384311-1.81195639e-03j 0.        +0.00000000e+00j
  0.47876121+6.19555525e-17j]]


In [74]:
# state_fidelity(partial_trace(rho3, 1), partial_trace(rho6, 1))

In [75]:
rho6 = rho3

In [76]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E* vari[0]/2)@U_H)                                  # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E* vari[0])@U_H)                                     # for tau
rho7=U_e2@rho6@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[1])):                                                # N과 tau를 N개 생성
    rho7 = U_e@np.kron(U180xp,I) @ rho7 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho8 = U_e2 @ np.kron(U180xp,I) @ rho7 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

# df8 = pd.DataFrame(rho8)
# print(df8)
print(partial_trace(rho8, 1))

[[5.00000000e-01-1.12185880e-16j 0.00000000e+00+0.00000000e+00j
  2.04523113e-17-2.27046219e-16j]
 [0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
 [6.76849057e-17+1.87997329e-16j 0.00000000e+00+0.00000000e+00j
  5.00000000e-01-2.00838646e-16j]]


In [77]:
irho_Z

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 1]])

In [78]:
# df8.to_csv('C:/Users/Administrator/git_zorocrit/qhackathon-2023/matrix/' + printdate + '_compare.csv',index=False)

In [79]:
state_fidelity(partial_trace(rho8, 1), irho_Z)

0.500000000000013